<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness1/63_PPO_Fraksi2_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 2 dijalankan menggunakan email shintaroudlotulhanafia@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [ ]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-ilb9d3l0
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-ilb9d3l0
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-o3xlt8a2/pyfolio_2de27ffb78d141b48c8fe4ec334f3f7a
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-o3xlt8a2/pyfolio_2de27ffb78d141b48c8fe4ec334f3f7a
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-o3xlt8a2/elegantrl_476cdd883ccd474f8ecaaa09c2c0f3ca
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-o3xlt8a2/elegantrl_476cdd883ccd474f8ecaaa09c2c0f3ca
     |████████████████████████████████| 2.3 MB 6.6 MB/s 
     |████████████████████████████████| 234 kB 67.0 MB/s 
     |██████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [ ]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [ ]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [ ]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi2).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10912, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [ ]:
df.shape

(10912, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [ ]:
df.nunique()

date      2728
open       874
high       895
low        893
close     1965
volume    4913
tic          4
day          5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [ ]:
print(df['tic'].unique())

['BMTR.JK' 'BTON.JK' 'FORU.JK' 'GEMA.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [ ]:
df.describe()

,open,high,low,close,volume,day
count,10912.000000,10912.000000,10912.000000,10912.000000,1.091200e+04,10912.000000
mean,382.164099,388.800147,375.111808,364.485108,7.102164e+06,1.988636
std,511.401972,519.888488,501.571396,484.876019,3.073051e+07,1.410209
min,10.400000,10.600000,10.000000,8.267043,0.000000e+00,0.000000
25%,90.000000,91.000000,88.000000,79.111000,2.000000e+01,1.000000
50%,192.500000,197.500000,190.000000,188.637268,8.200000e+04,2.000000
75%,386.000000,396.000000,378.000000,372.034546,2.504250e+06,3.000000
max,2775.000000,2800.000000,2675.000000,2589.928467,1.459050e+09,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,185.00,188.00,180.00,165.136459,4,BMTR.JK,0
1,2009-01-05,83.75,83.75,83.75,79.111000,0,BTON.JK,0
2,2009-01-05,55.00,55.00,55.00,41.633144,10000,FORU.JK,0
3,2009-01-05,28.00,28.00,28.00,21.837469,15,GEMA.JK,0
4,2009-01-06,185.00,188.00,185.00,165.136459,362500,BMTR.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [ ]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10604.000000,10604.000000,10604.000000,10604.000000,1.060400e+04,10604.000000,1.060400e+04,10604.000000,10604.000000,10604.000000,10604.000000
mean,382.566121,389.228980,375.459114,364.841163,7.132616e+06,2.019238,5.021978e-01,52.419585,2.552561,18.181388,237.184318
std,511.875229,520.386966,501.977945,485.323107,3.095670e+07,1.398187,1.869625e+01,13.816647,121.232871,7.336007,12022.051662
min,10.400000,10.600000,10.000000,8.267043,0.000000e+00,0.000000,-1.792874e+02,0.000000,-1000.000000,9.140000,0.000000
25%,90.000000,91.000000,88.000000,79.111000,2.000000e+01,1.000000,-2.066766e+00,45.503230,-80.942965,13.280000,0.280590
50%,192.500000,197.500000,190.000000,188.637268,8.280000e+04,2.000000,-1.421085e-13,50.128527,-4.235807,15.990000,1.207393
75%,388.250000,398.000000,380.000000,372.720673,2.542250e+06,3.000000,2.262522e+00,56.562113,78.776756,20.629999,3.573714
max,2775.000000,2800.000000,2675.000000,2589.928467,1.459050e+09,4.000000,1.227895e+02,100.000000,1000.000000,56.650002,619080.792212


In [ ]:
processed.nunique()

date           2651
open            872
high            893
low             893
close          1948
volume         4808
tic               4
day               5
macd          10414
rsi_30         6831
cci_30         9682
vix            1441
turbulence     2396
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['BMTR.JK', 'BTON.JK', 'FORU.JK', 'GEMA.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [ ]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [ ]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'BMTR.JK'),
 ('2009-01-05', 'BTON.JK'),
 ('2009-01-05', 'FORU.JK'),
 ('2009-01-05', 'GEMA.JK'),
 ('2009-01-06', 'BMTR.JK'),
 ('2009-01-06', 'BTON.JK'),
 ('2009-01-06', 'FORU.JK'),
 ('2009-01-06', 'GEMA.JK'),
 ('2009-01-07', 'BMTR.JK'),
 ('2009-01-07', 'BTON.JK'),
 ('2009-01-07', 'FORU.JK'),
 ('2009-01-07', 'GEMA.JK'),
 ('2009-01-08', 'BMTR.JK'),
 ('2009-01-08', 'BTON.JK'),
 ('2009-01-08', 'FORU.JK'),
 ('2009-01-08', 'GEMA.JK'),
 ('2009-01-09', 'BMTR.JK'),
 ('2009-01-09', 'BTON.JK'),
 ('2009-01-09', 'FORU.JK'),
 ('2009-01-09', 'GEMA.JK'),
 ('2009-01-10', 'BMTR.JK'),
 ('2009-01-10', 'BTON.JK'),
 ('2009-01-10', 'FORU.JK'),
 ('2009-01-10', 'GEMA.JK'),
 ('2009-01-11', 'BMTR.JK'),
 ('2009-01-11', 'BTON.JK'),
 ('2009-01-11', 'FORU.JK'),
 ('2009-01-11', 'GEMA.JK'),
 ('2009-01-12', 'BMTR.JK'),
 ('2009-01-12', 'BTON.JK'),
 ('2009-01-12', 'FORU.JK'),
 ('2009-01-12', 'GEMA.JK'),
 ('2009-01-13', 'BMTR.JK'),
 ('2009-01-13', 'BTON.JK'),
 ('2009-01-13', 'FORU.JK'),
 ('2009-01-13', 'GEM

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [ ]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136459,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837469,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136459,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,GEMA.JK,320.00,348.00,320.00,343.139648,250300.0,2.0,-6.835574,41.046063,-124.265358,12.580000,5.054559
16000,2019-12-19,BMTR.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,BTON.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,FORU.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [ ]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136459,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837469,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136459,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038942
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [ ]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136459,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837469,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136459,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038942
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [ ]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136459,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837469,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136459,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038942
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136459,4.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837469,15.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136459,362500.0,1.0,0.000000,0.0,66.666667,38.560001,0.0
5,2009-01-06,BTON.JK,83.75,83.75,83.75,79.111000,0.0,1.0,0.000000,0.0,66.666667,38.560001,0.0
6,2009-01-06,FORU.JK,55.00,55.00,53.00,40.119213,19500.0,1.0,-0.033966,0.0,-66.666667,38.560001,0.0
7,2009-01-06,GEMA.JK,28.00,28.00,28.00,21.837469,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
8,2009-01-07,BMTR.JK,189.00,189.00,185.00,168.706985,4.0,2.0,0.110722,100.0,97.052448,43.389999,0.0
9,2009-01-07,BTON.JK,83.75,83.75,83.75,79.111000,12.0,2.0,0.000000,100.0,97.052448,43.389999,0.0


In [ ]:
processed_full.nunique()

date           2651
tic               4
open            872
high            893
low             893
close          1948
volume         4808
day               5
macd          10414
rsi_30         6831
cci_30         9682
vix            1441
turbulence     2396
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [ ]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10604 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10604 non-null  object 
 1   tic         10604 non-null  object 
 2   open        10604 non-null  float64
 3   high        10604 non-null  float64
 4   low         10604 non-null  float64
 5   close       10604 non-null  float64
 6   volume      10604 non-null  float64
 7   day         10604 non-null  float64
 8   macd        10604 non-null  float64
 9   rsi_30      10604 non-null  float64
 10  cci_30      10604 non-null  float64
 11  vix         10604 non-null  float64
 12  turbulence  10604 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [ ]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi2/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.0,188.0,180.0,165.136459,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.0,188.0,185.0,165.136459,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
8,2009-01-07,BMTR.JK,189.0,189.0,185.0,168.706985,4.0,2.0,0.110722,100.000000,97.052448,43.389999,0.000000
12,2009-01-08,BMTR.JK,189.0,189.0,189.0,168.706985,3.0,3.0,0.158367,100.000000,96.065917,42.560001,0.000000
16,2009-01-09,BMTR.JK,190.0,191.0,187.0,169.599640,4.0,4.0,0.216443,100.000000,82.246657,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,BMTR.JK,364.0,366.0,350.0,352.000000,23827100.0,3.0,-6.919294,47.160507,-51.528730,13.940000,7.298998
15976,2019-12-13,BMTR.JK,354.0,356.0,350.0,354.000000,44426200.0,4.0,-6.487879,47.585547,-55.534056,12.630000,5.522403
15988,2019-12-16,BMTR.JK,354.0,358.0,354.0,356.000000,79595400.0,0.0,-5.916396,48.018107,-44.595031,12.140000,1.615356
15992,2019-12-17,BMTR.JK,356.0,372.0,356.0,366.000000,120860800.0,1.0,-4.603509,50.146184,-17.458651,12.290000,7.038942


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
5,2009-01-06,BTON.JK,83.75,83.75,83.75,79.111,0.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
9,2009-01-07,BTON.JK,83.75,83.75,83.75,79.111,12.0,2.0,0.000000,100.000000,97.052448,43.389999,0.000000
13,2009-01-08,BTON.JK,83.75,83.75,83.75,79.111,12.0,3.0,0.000000,100.000000,96.065917,42.560001,0.000000
17,2009-01-09,BTON.JK,83.75,83.75,83.75,79.111,12.0,4.0,0.000000,100.000000,82.246657,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BTON.JK,202.00,204.00,202.00,202.000,7100.0,3.0,-0.565595,48.389744,19.565217,13.940000,7.298998
15977,2019-12-13,BTON.JK,206.00,208.00,202.00,202.000,2000.0,4.0,-0.474011,48.389744,59.158945,12.630000,5.522403
15989,2019-12-16,BTON.JK,199.00,199.00,199.00,199.000,700.0,0.0,-0.636172,46.652491,-93.881857,12.140000,1.615356
15993,2019-12-17,BTON.JK,204.00,210.00,200.00,210.000,28900.0,1.0,0.121522,53.046489,138.358459,12.290000,7.038942


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,FORU.JK,55.0,55.0,55.0,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
6,2009-01-06,FORU.JK,55.0,55.0,53.0,40.119213,19500.0,1.0,-0.033966,0.000000,-66.666667,38.560001,0.000000
10,2009-01-07,FORU.JK,56.0,56.0,56.0,42.390114,8000.0,2.0,0.027141,60.810851,92.263926,43.389999,0.000000
14,2009-01-08,FORU.JK,51.0,51.0,51.0,38.605282,107500.0,3.0,-0.082382,29.686098,-124.581645,42.560001,0.000000
18,2009-01-09,FORU.JK,53.0,53.0,53.0,40.119213,10000.0,4.0,-0.079918,41.975227,-35.381012,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,FORU.JK,105.0,107.0,103.0,105.000000,16000.0,3.0,2.893300,46.901921,-97.919217,13.940000,7.298998
15978,2019-12-13,FORU.JK,105.0,105.0,105.0,105.000000,0.0,4.0,1.636381,46.901921,-94.350962,12.630000,5.522403
15990,2019-12-16,FORU.JK,105.0,105.0,105.0,105.000000,0.0,0.0,0.632967,46.901921,-88.779574,12.140000,1.615356
15994,2019-12-17,FORU.JK,101.0,101.0,100.0,101.000000,1700.0,1.0,-0.479485,44.683401,-125.823302,12.290000,7.038942


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,GEMA.JK,28.0,28.0,28.0,21.837469,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
7,2009-01-06,GEMA.JK,28.0,28.0,28.0,21.837469,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
11,2009-01-07,GEMA.JK,27.6,27.6,27.6,21.525505,2125000.0,2.0,-0.009674,0.000000,-100.000000,43.389999,0.000000
15,2009-01-08,GEMA.JK,27.6,27.6,27.6,21.525505,0.0,3.0,-0.013837,0.000000,-66.666667,42.560001,0.000000
19,2009-01-09,GEMA.JK,27.6,27.6,27.6,21.525505,15.0,4.0,-0.015808,0.000000,-55.555556,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,GEMA.JK,334.0,334.0,334.0,329.335175,12000.0,3.0,-7.415713,25.750664,-238.794227,13.940000,7.298998
15979,2019-12-13,GEMA.JK,346.0,346.0,346.0,341.167603,1900.0,4.0,-7.364107,37.695269,-121.623979,12.630000,5.522403
15991,2019-12-16,GEMA.JK,340.0,340.0,340.0,335.251404,20000.0,0.0,-7.711701,34.799621,-145.127241,12.140000,1.615356
15995,2019-12-17,GEMA.JK,320.0,348.0,320.0,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038942


In [ ]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,18500.0,18800.0,18000.0,16513.645935,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,8375.0,8375.0,8375.0,7911.100006,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,5500.0,5500.0,5500.0,4163.314438,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,2800.0,2800.0,2800.0,2183.746910,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,18500.0,18800.0,18500.0,16513.645935,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,32000.0,34800.0,32000.0,34313.964844,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038942
15996,2019-12-18,BMTR.JK,36800.0,37000.0,35800.0,35800.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,21200.0,21200.0,18800.0,19800.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,10100.0,10100.0,10100.0,10100.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [ ]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1976


In [ ]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,GEMA.JK,19400.0,19400.0,19400.0,18483.360291,500.0,3.0,4.955438,85.663835,85.451856,10.18,0.768637
2156,2017-12-29,BMTR.JK,58000.0,60000.0,57500.0,58438.098145,47846400.0,4.0,0.700427,51.611650,50.420800,11.04,0.652399
2156,2017-12-29,BTON.JK,11500.0,11500.0,11300.0,11300.000000,72000.0,4.0,-0.668175,47.825672,-50.397878,11.04,0.652399
2156,2017-12-29,FORU.JK,12600.0,14300.0,12400.0,12500.000000,10800.0,4.0,-19.008832,36.861538,-119.781160,11.04,0.652399
2156,2017-12-29,GEMA.JK,19400.0,19400.0,19200.0,18483.360291,6134000.0,4.0,4.784416,85.663835,75.050421,11.04,0.652399


In [ ]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,BMTR.JK,58500.0,61000.0,58500.0,60419.049072,51308700.0,1.0,3.323453,54.022498,122.833904,9.77,5.289308
0,2018-01-02,BTON.JK,11300.0,11500.0,11100.0,11200.000000,72800.0,1.0,-0.763789,46.868249,-79.279279,9.77,5.289308
0,2018-01-02,FORU.JK,12000.0,12000.0,12000.0,12000.000000,100.0,1.0,-18.873467,35.964867,-122.764317,9.77,5.289308
0,2018-01-02,GEMA.JK,19200.0,20200.0,19200.0,19245.561218,1657500.0,1.0,5.003854,89.250602,121.394145,9.77,5.289308
1,2018-01-03,BMTR.JK,61000.0,61500.0,60000.0,60419.049072,51856600.0,2.0,4.876092,54.022498,148.499761,9.15,7.368367


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [ ]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [ ]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [ ]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 88307757.23
total_reward: -11692242.77
total_cost: 72273507.57
total_trades: 8356
Sharpe: 0.019
day: 2156, episode: 20
begin_total_asset: 100000000.00
end_total_asset: 125956060.59
total_reward: 25956060.59
total_cost: 76152689.71
total_trades: 8341
Sharpe: 0.232
-----------------------------------
| time/              |            |
|    fps             | 359        |
|    iterations      | 1          |
|    time_elapsed    | 138        |
|    total_timesteps | 50000      |
| train/             |            |
|    reward          | -114.74172 |
-----------------------------------


<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [ ]:
insample_risk_indicator.turbulence.describe()

count      2157.000000
mean        290.374792
std       13329.687343
min           0.000000
25%           0.250728
50%           1.145774
75%           3.447183
max      619080.792212
Name: turbulence, dtype: float64

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

60.36355428011014

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,BMTR.JK,58500.0,61000.0,58500.0,60419.049072,51308700.0,1.0,3.323453,54.022498,122.833904,9.77,5.289308
0,2018-01-02,BTON.JK,11300.0,11500.0,11100.0,11200.000000,72800.0,1.0,-0.763789,46.868249,-79.279279,9.77,5.289308
0,2018-01-02,FORU.JK,12000.0,12000.0,12000.0,12000.000000,100.0,1.0,-18.873467,35.964867,-122.764317,9.77,5.289308
0,2018-01-02,GEMA.JK,19200.0,20200.0,19200.0,19245.561218,1657500.0,1.0,5.003854,89.250602,121.394145,9.77,5.289308
1,2018-01-03,BMTR.JK,61000.0,61500.0,60000.0,60419.049072,51856600.0,2.0,4.876092,54.022498,148.499761,9.15,7.368367


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

In [ ]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [ ]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [ ]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_ppo.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [ ]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,BMTR.JK,BTON.JK,FORU.JK,GEMA.JK,total_shares,account_value
0,2018-01-02,0.0,28.0,8.0,0.0,36.0,1.000000e+08
1,2018-01-03,0.0,28.0,8.0,0.0,72.0,9.999906e+07
2,2018-01-04,0.0,28.0,8.0,0.0,108.0,1.000005e+08
3,2018-01-05,0.0,28.0,8.0,0.0,144.0,1.000200e+08
4,2018-01-08,0.0,28.0,8.0,0.0,180.0,1.000350e+08
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,0.0,0.0,0.0,5546.0,1.019326e+08
490,2019-12-13,0.0,0.0,0.0,0.0,5546.0,1.019326e+08
491,2019-12-16,0.0,0.0,0.0,0.0,5546.0,1.005811e+08
492,2019-12-17,0.0,0.0,0.0,0.0,5546.0,1.051202e+08


In [ ]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi2/df_actions_and_account_value_ppo(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.2. Agen 2: PPO

In [ ]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi2/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,BMTR.JK,BTON.JK,FORU.JK,GEMA.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 28.0 saham,Beli 8.0 saham,Tahan,36.0,1.000000e+08
1,2018-01-03,Tahan,Beli 28.0 saham,Beli 8.0 saham,Tahan,72.0,9.999906e+07
2,2018-01-04,Tahan,Beli 28.0 saham,Beli 8.0 saham,Tahan,108.0,1.000005e+08
3,2018-01-05,Tahan,Beli 28.0 saham,Beli 8.0 saham,Tahan,144.0,1.000200e+08
4,2018-01-08,Tahan,Beli 28.0 saham,Beli 8.0 saham,Tahan,180.0,1.000350e+08
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Tahan,Tahan,Tahan,5546.0,1.019326e+08
490,2019-12-13,Tahan,Tahan,Tahan,Tahan,5546.0,1.019326e+08
491,2019-12-16,Tahan,Tahan,Tahan,Tahan,5546.0,1.005811e+08
492,2019-12-17,Tahan,Tahan,Tahan,Tahan,5546.0,1.051202e+08


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [ ]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi2/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,BMTR.JK,51308700.0,1.0,3.323453,54.022498,122.833904,9.77,5.289308,0.0
13140,2018-01-03,BMTR.JK,51856600.0,2.0,4.876092,54.022498,148.499761,9.15,7.368367,0.0
13144,2018-01-04,BMTR.JK,50366900.0,3.0,6.432041,54.627297,145.027224,9.22,3.277982,0.0
13148,2018-01-05,BMTR.JK,53726900.0,4.0,6.392601,52.484687,112.730735,9.22,23.155409,0.0
13160,2018-01-08,BMTR.JK,63843600.0,0.0,9.844406,57.640823,214.501064,9.52,8.314841,0.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,BMTR.JK,23827100.0,3.0,-6.919294,47.160507,-51.528730,13.94,7.298998,0.0
15976,2019-12-13,BMTR.JK,44426200.0,4.0,-6.487879,47.585547,-55.534056,12.63,5.522403,0.0
15988,2019-12-16,BMTR.JK,79595400.0,0.0,-5.916396,48.018107,-44.595031,12.14,1.615356,0.0
15992,2019-12-17,BMTR.JK,120860800.0,1.0,-4.603509,50.146184,-17.458651,12.29,7.038942,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BTON.JK,72800.0,1.0,-0.763789,46.868249,-79.279279,9.77,5.289308,28.0
13141,2018-01-03,BTON.JK,28400.0,2.0,-0.979169,45.004234,-139.118457,9.15,7.368367,28.0
13145,2018-01-04,BTON.JK,600.0,3.0,-1.056984,46.112770,-131.652661,9.22,3.277982,28.0
13149,2018-01-05,BTON.JK,143700.0,4.0,-1.026133,47.213465,-16.601563,9.22,23.155409,28.0
13161,2018-01-08,BTON.JK,266700.0,0.0,-0.830724,49.353798,-26.898734,9.52,8.314841,28.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BTON.JK,7100.0,3.0,-0.565595,48.389744,19.565217,13.94,7.298998,0.0
15977,2019-12-13,BTON.JK,2000.0,4.0,-0.474011,48.389744,59.158945,12.63,5.522403,0.0
15989,2019-12-16,BTON.JK,700.0,0.0,-0.636172,46.652491,-93.881857,12.14,1.615356,0.0
15993,2019-12-17,BTON.JK,28900.0,1.0,0.121522,53.046489,138.358459,12.29,7.038942,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,FORU.JK,100.0,1.0,-18.873467,35.964867,-122.764317,9.77,5.289308,8.0
13142,2018-01-03,FORU.JK,21700.0,2.0,-18.134343,38.144045,-100.866533,9.15,7.368367,8.0
13146,2018-01-04,FORU.JK,1600.0,3.0,-17.508142,37.764197,-91.342149,9.22,3.277982,8.0
13150,2018-01-05,FORU.JK,20500.0,4.0,-16.419146,39.326781,-72.042507,9.22,23.155409,8.0
13162,2018-01-08,FORU.JK,2800.0,0.0,-15.538375,38.922404,-74.684721,9.52,8.314841,8.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,FORU.JK,16000.0,3.0,2.893300,46.901921,-97.919217,13.94,7.298998,0.0
15978,2019-12-13,FORU.JK,0.0,4.0,1.636381,46.901921,-94.350962,12.63,5.522403,0.0
15990,2019-12-16,FORU.JK,0.0,0.0,0.632967,46.901921,-88.779574,12.14,1.615356,0.0
15994,2019-12-17,FORU.JK,1700.0,1.0,-0.479485,44.683401,-125.823302,12.29,7.038942,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,GEMA.JK,1657500.0,1.0,5.003854,89.250602,121.394145,9.77,5.289308,0.0
13143,2018-01-03,GEMA.JK,31000.0,2.0,6.026481,91.878189,247.409226,9.15,7.368367,0.0
13147,2018-01-04,GEMA.JK,194500.0,3.0,7.367029,93.244742,278.509576,9.22,3.277982,0.0
13151,2018-01-05,GEMA.JK,3254500.0,4.0,9.853419,95.292990,399.676278,9.22,23.155409,0.0
13163,2018-01-08,GEMA.JK,3336500.0,0.0,11.689155,95.292990,344.885597,9.52,8.314841,0.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,GEMA.JK,12000.0,3.0,-7.415713,25.750664,-238.794227,13.94,7.298998,0.0
15979,2019-12-13,GEMA.JK,1900.0,4.0,-7.364107,37.695269,-121.623979,12.63,5.522403,0.0
15991,2019-12-16,GEMA.JK,20000.0,0.0,-7.711701,34.799621,-145.127241,12.14,1.615356,0.0
15995,2019-12-17,GEMA.JK,224900.0,1.0,-7.266888,41.046063,-139.035782,12.29,7.038942,0.0


####String Action Version

In [ ]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi2/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,BMTR.JK
0,13136,2018-01-02,51308700.0,1.0,3.323453,54.022498,122.833904,9.77,5.289308,Tahan
1,13140,2018-01-03,51856600.0,2.0,4.876092,54.022498,148.499761,9.15,7.368367,Tahan
2,13144,2018-01-04,50366900.0,3.0,6.432041,54.627297,145.027224,9.22,3.277982,Tahan
3,13148,2018-01-05,53726900.0,4.0,6.392601,52.484687,112.730735,9.22,23.155409,Tahan
4,13160,2018-01-08,63843600.0,0.0,9.844406,57.640823,214.501064,9.52,8.314841,Tahan
...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,23827100.0,3.0,-6.919294,47.160507,-51.528730,13.94,7.298998,Tahan
490,15976,2019-12-13,44426200.0,4.0,-6.487879,47.585547,-55.534056,12.63,5.522403,Tahan
491,15988,2019-12-16,79595400.0,0.0,-5.916396,48.018107,-44.595031,12.14,1.615356,Tahan
492,15992,2019-12-17,120860800.0,1.0,-4.603509,50.146184,-17.458651,12.29,7.038942,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,BTON.JK
0,13137,2018-01-02,72800.0,1.0,-0.763789,46.868249,-79.279279,9.77,5.289308,Beli 28.0 saham
1,13141,2018-01-03,28400.0,2.0,-0.979169,45.004234,-139.118457,9.15,7.368367,Beli 28.0 saham
2,13145,2018-01-04,600.0,3.0,-1.056984,46.112770,-131.652661,9.22,3.277982,Beli 28.0 saham
3,13149,2018-01-05,143700.0,4.0,-1.026133,47.213465,-16.601563,9.22,23.155409,Beli 28.0 saham
4,13161,2018-01-08,266700.0,0.0,-0.830724,49.353798,-26.898734,9.52,8.314841,Beli 28.0 saham
...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,7100.0,3.0,-0.565595,48.389744,19.565217,13.94,7.298998,Tahan
490,15977,2019-12-13,2000.0,4.0,-0.474011,48.389744,59.158945,12.63,5.522403,Tahan
491,15989,2019-12-16,700.0,0.0,-0.636172,46.652491,-93.881857,12.14,1.615356,Tahan
492,15993,2019-12-17,28900.0,1.0,0.121522,53.046489,138.358459,12.29,7.038942,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,FORU.JK
0,13138,2018-01-02,100.0,1.0,-18.873467,35.964867,-122.764317,9.77,5.289308,Beli 8.0 saham
1,13142,2018-01-03,21700.0,2.0,-18.134343,38.144045,-100.866533,9.15,7.368367,Beli 8.0 saham
2,13146,2018-01-04,1600.0,3.0,-17.508142,37.764197,-91.342149,9.22,3.277982,Beli 8.0 saham
3,13150,2018-01-05,20500.0,4.0,-16.419146,39.326781,-72.042507,9.22,23.155409,Beli 8.0 saham
4,13162,2018-01-08,2800.0,0.0,-15.538375,38.922404,-74.684721,9.52,8.314841,Beli 8.0 saham
...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,16000.0,3.0,2.893300,46.901921,-97.919217,13.94,7.298998,Tahan
490,15978,2019-12-13,0.0,4.0,1.636381,46.901921,-94.350962,12.63,5.522403,Tahan
491,15990,2019-12-16,0.0,0.0,0.632967,46.901921,-88.779574,12.14,1.615356,Tahan
492,15994,2019-12-17,1700.0,1.0,-0.479485,44.683401,-125.823302,12.29,7.038942,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,GEMA.JK
0,13139,2018-01-02,1657500.0,1.0,5.003854,89.250602,121.394145,9.77,5.289308,Tahan
1,13143,2018-01-03,31000.0,2.0,6.026481,91.878189,247.409226,9.15,7.368367,Tahan
2,13147,2018-01-04,194500.0,3.0,7.367029,93.244742,278.509576,9.22,3.277982,Tahan
3,13151,2018-01-05,3254500.0,4.0,9.853419,95.292990,399.676278,9.22,23.155409,Tahan
4,13163,2018-01-08,3336500.0,0.0,11.689155,95.292990,344.885597,9.52,8.314841,Tahan
...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,12000.0,3.0,-7.415713,25.750664,-238.794227,13.94,7.298998,Tahan
490,15979,2019-12-13,1900.0,4.0,-7.364107,37.695269,-121.623979,12.63,5.522403,Tahan
491,15991,2019-12-16,20000.0,0.0,-7.711701,34.799621,-145.127241,12.14,1.615356,Tahan
492,15995,2019-12-17,224900.0,1.0,-7.266888,41.046063,-139.035782,12.29,7.038942,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.2 Agen ppo

In [ ]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return         -0.001459
Cumulative returns    -0.002858
Annual volatility      0.448251
Sharpe ratio           0.217245
Calmar ratio          -0.003695
Stability              0.002005
Max drawdown          -0.394860
Omega ratio            1.050420
Sortino ratio          0.337904
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.013216
Daily value at risk   -0.056088
dtype: float64


###8.1.6. Baseline Stats

In [ ]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_ppo.loc[0,'date'],
        end = df_account_value_ppo.loc[len(df_account_value_ppo)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64


In [ ]:
df_account_value_ppo.loc[0,'date']

'2018-01-02'

In [ ]:
df_account_value_ppo.loc[len(df_account_value_a2c)-1,'date']

NameError: ignored

<a id='7.2'></a>
## 8.2 Gambaran BackTesting

###8.2.2. Agen PPO

In [ ]:
print("===========PPO Compare to JII===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
# JII: ^JKII
backtest_plot(df_account_value_ppo, 
             baseline_ticker = '^JKII', 
             baseline_start = df_account_value_ppo.loc[0,'date'],
             baseline_end = df_account_value_ppo.loc[len(df_account_value_ppo)-1,'date'])